https://github.com/tensorflow/tensorflow/blob/r1.13/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import unicodedata
import re
import numpy as np
import os
import time

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


In [6]:
tf.enable_eager_execution()

In [7]:
path_to_file = './data/cmn-eng/cmn.txt'

In [47]:
def preprocess_sentence(s):
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?。.!,¿])", r" \1 ", s)
    w = re.sub(r'[" "]+', " ", w)
    
    # a bit clean up
    w = re.sub(r'[^a-zA-Z?.!,。\u4E00-\u9FFF]', ' ', w)
    
    w = w.strip().lower()
    w = '<start> ' + w + ' <end>'
    return w

In [48]:
def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    return word_pairs

In [106]:
class LanguageIndex():
    def __init__(self, lang, split=None):
        """
        :param lang: A list of sentences
        :param split: How to split sentence into words
        """
        self.word2idx = dict()
        self.idx2word = dict()
        self.vocab = set()
        
        self.create_index(lang, split)
        
    def create_index(self, lang, split):
        for phrase in lang:
            if split is not None:
                phrase = phrase.split(split)
                self.vocab.update(phrase)
            else:
                for p in phrase.split(' '):
                    if p in ('<start>', '<end>') or len(p)==1:
                        self.vocab.add(p)
                    else:
                        self.vocab.update(p)
        
        self.vocab = sorted(self.vocab)
        self.word2idx['<pad>'] = 0
        
        for index, word in enumerate(self.vocab):
              self.word2idx[word] = index + 1
    
        for word, index in self.word2idx.items():
              self.idx2word[index] = word
                
def max_length(tensor):
    return max(len(t) for t in tensor)

In [113]:
num_examples = 50000

pairs = create_dataset(path_to_file, num_examples)
inp_lang = LanguageIndex([en for en, ch in pairs], ' ')
targ_lang = LanguageIndex([ch for en, ch in pairs], None)

In [114]:
def split_chinese(s):
    res = list()
    for p in s.split(' '):
        if p in ('<start>', '<end>') or len(p)==1:
            res.append(p)
        else:
            res.extend(p)
    return res

In [115]:
# english sentences
input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, _ in pairs]

# Chinses sentences
target_tensor = [[targ_lang.word2idx[s] for s in split_chinese(ch)] for _, ch in pairs]

# Calculate max_length of input and output tensor
# Here, we'll set those to the longest sentence in the dataset
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

# Padding the input and output tensor to the maximum length
input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                             maxlen=max_length_inp,
                                                             padding='post')

target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                              maxlen=max_length_tar, 
                                                              padding='post')

In [116]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(18362, 18362, 2041, 2041)

#### Create dataset

In [118]:
BUFFER_SIZE = 15000
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

#### Build Model

In [119]:
def gru(units):
    return tf.keras.layers.GRU(units,
                              return_sequences=True,
                              return_state=True,
                              recurrent_activation='sigmoid',
                              recurrent_initializer='glorot_uniform')

In [120]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [132]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        score = self.V(tf.nn.tanh(
            self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        attention_weights = tf.nn.softmax(score, axis=1)
        
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        output, state = self.gru(x)
        
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [142]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [137]:
optimizer = tf.train.AdamOptimizer()

def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

#### Checkpoint (Object-based saving)

In [138]:
checkpoint_dir = './checkpoints/'
checkpoint_prefix = os.path.join(checkpoint_dir, "nmt-keras")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

#### Training

In [143]:
EPOCHS = 6

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch == 1:
            print('Time taken for 1 batch {} sec\n'.format(time.time() - start))

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
        
           
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.9742
Time taken for 1 batch 34.034992933273315 sec

Epoch 1 Batch 100 Loss 1.2317
Epoch 1 Batch 200 Loss 1.1246


KeyboardInterrupt: 